In [1]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from astropy import constants as const
import matplotlib as mpl
from jupyterthemes import jtplot #These two lines can be skipped if you are not using jupyter themes
jtplot.reset()

from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=67.4, Om0=0.314)
import scipy as sp
import multiprocessing as mp

from matplotlib.ticker import StrMethodFormatter
import time
start_total = time.time()

In [2]:
import os
my_path = '/home/tomi/Documentos/Fisica/Tesis/escrito-tesis/images/'

In [3]:
from lenstronomy.LensModel.lens_model import LensModel
from lenstronomy.LensModel.lens_model_extensions import LensModelExtensions
from lenstronomy.LensModel.Solver.lens_equation_solver import LensEquationSolver

In [4]:
zl = 0.2; zs = 1.2
Dl = cosmo.angular_diameter_distance(zl)   
Ds = cosmo.angular_diameter_distance(zs)    
Dls = cosmo.angular_diameter_distance_z1z2(zl, zs)
G = const.G
rho_crit = (cosmo.critical_density(zl)).to(u.kg/u.m**3)
c_light = (const.c).to(u.cm/u.second)

#r0 = 10*u.kpc
r0 = 10.0*u.kpc
#r0 = 0.1*u.kpc
pi = np.pi

def scale_radius(v,Dl,Ds,Dls):                               #this is e0 in eq 3.42 meneghetti, eq 1 barnacka 2014
    return (4.*pi*v**2/c_light**2*Dl*Dls/Ds).decompose()
def theta_E_SIS():
    'in arcsec'
    pre_theta_E = (scale_radius(v,Dl,Ds,Dls)/Dl).decompose()
    return pre_theta_E*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 

v = 180 *u.km/u.s
ss_r = scale_radius(v,Dl,Ds,Dls) 
print('scale radius (m): ',ss_r)
print('scale radius (kpc): ',ss_r.to(u.kpc))
print('theta_E: ',theta_E_SIS() ,'arcsec')
theta_E_num = theta_E_SIS()
re = (const.e.esu**2/const.m_e/(c_light**2)).decompose()
print('Classic electron radius: ',re)

scale radius (m):  7.705329461274929e+19 m
scale radius (kpc):  2.49712721364453 kpc
theta_E:  0.7301786792241515 arcsec
Classic electron radius:  2.817940324670788e-15 m


In [5]:
nu = 130.0 * u.MHz
omega = nu*(1.+zl)*2*np.pi
n0 = 300./(u.cm)**(3)

In [6]:
theta_0 = (r0/Dl).decompose()*u.rad.to('arcsec', equivalencies=u.dimensionless_angles()) 
print('Angular scale radius (theta_0 = r0/Dl): ',theta_0, 'arcsec')

Angular scale radius (theta_0 = r0/Dl):  2.9240748137875747 arcsec


In [7]:
from lenstronomy.LensModel.Profiles.plasma_exponential_sie import PlasmaExponentialSIE
perfil = PlasmaExponentialSIE()

In [8]:
def image_position_3(x, y, eta):
    x_s, y_s = x, y
    x_pre, y_pre = lensEquationSolver.image_position_from_source(kwargs_lens=kwargs_lens_list, \
                    sourcePos_x=x_s, sourcePos_y=y_s, min_distance=0.05, search_window=5, \
                    precision_limit=10**(-8), num_iter_max=100,non_linear=True)
    print(x_pre)
    print(y_pre)
    mag = [0,0]
    t = [0,0]
    for k in range(len(x_pre)):
        mag[k] = lensModel.magnification(x_pre[k], y_pre[k], kwargs = kwargs_lens_list)
        phi = PlasmaExponentialSIE.function(perfil, x_pre[k], y_pre[k], theta_E_num.value, eta, A, B, C ,psi0_plasma.value, theta_0.value)
        t[k] = ((1+zl)/c_light*Ds*Dl/Dls*( 1/2*( (x_s - x_pre[k] )**2 + (y_s - y_pre[k])**2) - phi**2)*(u.arcsec**2).to('rad**2')).to('s').value
    r = np.abs(mag[0]/mag[1])
    rr = [r,r]
    dt = np.abs(t[0] - t[1])/24/60/60 #convert seconds to days
    dt = [dt,dt]
    return x_pre, y_pre, rr, dt

In [9]:
lens_model_list = ['PlasmaExponentialSIE']
lensModel = LensModel(lens_model_list)
lensEquationSolver = LensEquationSolver(lensModel)
A = 2.00744300029693
B = 1.5887225542022732
C = 1.4308204791467736
psi0_plasma = (A*Dls/Ds/Dl*2*pi*c_light**2/omega**2*re*n0*r0 \
            *(u.rad**2).to('arcsec**2', equivalencies=u.dimensionless_angles()) ).decompose()

In [10]:
eta_arr = np.linspace(.05, .39, 50)
r_arr = np.linspace(.3, 1, 50)
x_arr = y_arr = r_arr*np.cos(np.pi/4)

In [ ]:
x_line_arr = []
y_line_arr = []
result = []
for x in x_arr:
    for i in eta_arr:
        kwargs = {'theta_E':theta_E_num.value,'eta':i, 'A':A, 'B':B, 'C':C ,'psi0_plasma':psi0_plasma.value,\
                        'theta_0':theta_0.value}
        kwargs_lens_list = [kwargs]
        x_image, y_image, mag, dt = image_position_3(x, x, i)
        tmp = [x, i, mag[0], dt[0]]
        result.append(tmp)
        print(x,i)

[ 0.59804034 -0.29882478]
[ 0.7587761 -0.2348826]
0.21213203435596426 0.05
[ 0.5868185  -0.30200279]
[ 0.76825987 -0.22977376]
0.21213203435596426 0.056938775510204084
[ 0.57579995 -0.30497097]
[ 0.77740342 -0.22466496]
0.21213203435596426 0.06387755102040817
[ 0.56500968 -0.30772785]
[ 0.78620581 -0.21957027]
0.21213203435596426 0.07081632653061225
[ 0.55446803 -0.31027343]
[ 0.79467005 -0.21450264]
0.21213203435596426 0.07775510204081633
[ 0.54419083 -0.31260906]
[ 0.80280241 -0.20947379]
0.21213203435596426 0.08469387755102042
[ 0.53418978 -0.31473722]
[ 0.81061178 -0.20449425]
0.21213203435596426 0.0916326530612245
[ 0.52447282 -0.31666143]
[ 0.81810907 -0.19957328]
0.21213203435596426 0.09857142857142859
[ 0.5150446  -0.31838602]
[ 0.82530663 -0.19471895]
0.21213203435596426 0.10551020408163267
[ 0.50590689 -0.31991601]
[ 0.83221779 -0.18993815]
0.21213203435596426 0.11244897959183675
[ 0.49705905 -0.32125697]
[ 0.83885643 -0.18523669]
0.21213203435596426 0.11938775510204083
[ 0.4

[ 0.34328563 -0.28800162]
[ 0.97724675 -0.08234695]
0.22223355980148637 0.33448979591836736
[ 0.34007242 -0.28609725]
[ 0.98020468 -0.08021014]
0.22223355980148637 0.34142857142857147
[ 0.33694496 -0.28414231]
[ 0.98312292 -0.07812318]
0.22223355980148637 0.3483673469387755
[ 0.33390001 -0.28213822]
[ 0.98600332 -0.07608465]
0.22223355980148637 0.35530612244897963
[ 0.33093452 -0.28008633]
[ 0.98884764 -0.07409316]
0.22223355980148637 0.3622448979591837
[ 0.32804556 -0.27798786]
[ 0.99165753 -0.07214737]
0.22223355980148637 0.3691836734693878
[ 0.32523034 -0.27584398]
[ 0.99443454 -0.07024599]
0.22223355980148637 0.37612244897959185
[ 0.32248617 -0.27365577]
[ 0.99718011 -0.06838774]
0.22223355980148637 0.38306122448979596
[ 0.31981051 -0.27142425]
[ 0.99989561 -0.0665714 ]
0.22223355980148637 0.39
[ 0.62287856 -0.27539724]
[ 0.77462106 -0.22094006]
0.2323350852470085 0.05
[ 0.61225771 -0.27814584]
[ 0.78375953 -0.21655955]
0.2323350852470085 0.056938775510204084
[ 0.60179875 -0.280724

[ 0.40463106 -0.27752539]
[ 0.96501583 -0.10171552]
0.24243661069253059 0.27204081632653065
[ 0.40035971 -0.27624828]
[ 0.968534   -0.09917511]
0.24243661069253059 0.2789795918367347
[ 0.3962086  -0.27490362]
[ 0.9719867 -0.0966906]
0.24243661069253059 0.2859183673469388
[ 0.39217325 -0.27349367]
[ 0.97537729 -0.0942608 ]
0.24243661069253059 0.29285714285714287
[ 0.38824937 -0.27202056]
[ 0.97870892 -0.0918845 ]
0.24243661069253059 0.299795918367347
[ 0.38443283 -0.2704863 ]
[ 0.98198454 -0.08956052]
0.24243661069253059 0.30673469387755103
[ 0.38071966 -0.26889281]
[ 0.98520693 -0.08728763]
0.24243661069253059 0.31367346938775514
[ 0.37710608 -0.26724189]
[ 0.98837869 -0.08506466]
0.24243661069253059 0.3206122448979592
[ 0.37358842 -0.26553527]
[ 0.99150227 -0.08289041]
0.24243661069253059 0.3275510204081633
[ 0.37016322 -0.26377455]
[ 0.99457995 -0.0807637 ]
0.24243661069253059 0.33448979591836736
[ 0.36682712 -0.26196125]
[ 0.9976139  -0.07868337]
0.24243661069253059 0.34142857142857

[ 0.46620415 -0.25908302]
[ 0.95425617 -0.11625426]
0.2626396615835748 0.22346938775510206
[ 0.46088994 -0.25840311]
[ 0.95843862 -0.11349898]
0.2626396615835748 0.23040816326530617
[ 0.45572546 -0.2576411 ]
[ 0.96252497 -0.11079519]
0.2626396615835748 0.23734693877551022
[ 0.45070577 -0.25679956]
[ 0.96652016 -0.10814258]
0.2626396615835748 0.24428571428571433
[ 0.44582602 -0.255881  ]
[ 0.97042886 -0.10554077]
0.2626396615835748 0.2512244897959184
[ 0.44108147 -0.25488785]
[ 0.97425545 -0.10298929]
0.2626396615835748 0.2581632653061225
[ 0.4364675  -0.25382242]
[ 0.97800409 -0.10048758]
0.2626396615835748 0.26510204081632655
[ 0.43197961 -0.25268697]
[ 0.98167868 -0.09803504]
0.2626396615835748 0.27204081632653065
[ 0.42761346 -0.25148365]
[ 0.98528293 -0.09563099]
0.2626396615835748 0.2789795918367347
[ 0.42336482 -0.2502145 ]
[ 0.98882031 -0.09327472]
0.2626396615835748 0.2859183673469388
[ 0.41922959 -0.2488815 ]
[ 0.99229408 -0.09096549]
0.2626396615835748 0.29285714285714287
[ 0

0.282842712474619 0.1679591836734694
[ 0.53528004 -0.23638234]
[ 0.93755075 -0.12894763]
0.282842712474619 0.17489795918367348
[ 0.52880545 -0.23633457]
[ 0.94262663 -0.12613559]
0.282842712474619 0.18183673469387757
[ 0.52250254 -0.23619431]
[ 0.94756873 -0.12335999]
0.282842712474619 0.18877551020408168
[ 0.51636756 -0.23596379]
[ 0.95238318 -0.120622  ]
0.282842712474619 0.19571428571428573
[ 0.51039655 -0.23564526]
[ 0.95707592 -0.11792257]
0.282842712474619 0.20265306122448984
[ 0.50458543 -0.23524097]
[ 0.96165268 -0.11526251]
0.282842712474619 0.2095918367346939
[ 0.49893    -0.23475316]
[ 0.96611898 -0.11264245]
0.282842712474619 0.216530612244898
[ 0.49342599 -0.23418404]
[ 0.97048007 -0.11006287]
0.282842712474619 0.22346938775510206
[ 0.48806913 -0.2335358 ]
[ 0.97474101 -0.10752412]
0.282842712474619 0.23040816326530617
[ 0.48285511 -0.23281058]
[ 0.97890661 -0.10502643]
0.282842712474619 0.23734693877551022
[ 0.47777968 -0.23201048]
[ 0.98298145 -0.10256992]
0.282842712474

...

[0.66618958]
[1.24734497]
0.5050762722761053 0.37612244897959185
[0.66251936]
[1.25074134]
0.5050762722761053 0.38306122448979596
[0.65890969]
[1.25409811]
0.5050762722761053 0.39
[ 9.31174091e-01 -1.33548513e-05]
[ 1.02878105e+00 -1.20818127e-05]
0.5151777977216275 0.05
[ 9.24345037e-01 -4.37091455e-06]
[ 1.03527731e+00 -3.89972314e-06]
0.5151777977216275 0.056938775510204084
[ 9.17538482e-01 -2.24500892e-07]
[ 1.04171309e+00 -1.97537833e-07]
0.5151777977216275 0.06387755102040817
[0.91075957]
[1.04808503]
0.5151777977216275 0.07081632653061225
[0.90401328]
[1.05439006]
0.5151777977216275 0.07775510204081633
[0.89730436]
[1.06062544]
0.5151777977216275 0.08469387755102042
[0.89063737]
[1.06678874]
0.5151777977216275 0.0916326530612245
[0.88401661]
[1.07287785]
0.5151777977216275 0.09857142857142859
[0.87744613]
[1.07889097]
0.5151777977216275 0.10551020408163267
[0.87092972]
[1.08482658]
0.5151777977216275 0.11244897959183675
[0.86447091]
[1.09068349]
0.5151777977216275 0.119387755102

[0.80433811]
[1.18238959]
0.5353808486126718 0.216530612244898
[0.79901944]
[1.18707308]
0.5353808486126718 0.22346938775510206
[0.79377732]
[1.19168798]
0.5353808486126718 0.23040816326530617
[0.7886117]
[1.19623554]
0.5353808486126718 0.23734693877551022
[0.78352238]
[1.20071704]
0.5353808486126718 0.24428571428571433
[0.77850908]
[1.20513378]
0.5353808486126718 0.2512244897959184
[0.7735714]
[1.2094871]
0.5353808486126718 0.2581632653061225
[0.76870885]
[1.21377836]
0.5353808486126718 0.26510204081632655
[0.76392089]
[1.21800892]
0.5353808486126718 0.27204081632653065
[0.75920686]
[1.22218014]
0.5353808486126718 0.2789795918367347
[0.75456608]
[1.2262934]
0.5353808486126718 0.2859183673469388
[0.74999777]
[1.23035006]
0.5353808486126718 0.29285714285714287
[0.74550114]
[1.23435148]
0.5353808486126718 0.299795918367347
[0.74107532]
[1.238299]
0.5353808486126718 0.30673469387755103
[0.73671941]
[1.24219396]
0.5353808486126718 0.31367346938775514
[0.7324325]
[1.24603767]
0.535380848612

[0.95093194]
[1.10768614]
0.5656854249492381 0.08469387755102042
[0.94452725]
[1.11366754]
0.5656854249492381 0.0916326530612245
[0.93816076]
[1.11958624]
0.5656854249492381 0.09857142857142859
[0.93183597]
[1.12544048]
0.5656854249492381 0.10551020408163267
[0.92555618]
[1.13122875]
0.5656854249492381 0.11244897959183675
[0.9193245]
[1.13694974]
0.5656854249492381 0.11938775510204083
[0.91314382]
[1.14260238]
0.5656854249492381 0.1263265306122449
[0.90701685]
[1.1481858]
0.5656854249492381 0.133265306122449
[0.90094607]
[1.15369934]
0.5656854249492381 0.14020408163265308
[0.89493374]
[1.15914253]
0.5656854249492381 0.14714285714285716
[0.88898196]
[1.16451509]
0.5656854249492381 0.15408163265306124
[0.88309259]
[1.1698169]
0.5656854249492381 0.16102040816326532
[0.87726729]
[1.175048]
0.5656854249492381 0.1679591836734694
[0.87150755]
[1.18020858]
0.5656854249492381 0.17489795918367348
[0.86581465]
[1.18529898]
0.5656854249492381 0.18183673469387757
[0.86018973]
[1.19031963]
0.5656854

[0.78091609]
[1.29975022]
0.5858884758402823 0.33448979591836736
[0.7767963]
[1.30350239]
0.5858884758402823 0.34142857142857147
[0.77273742]
[1.30720977]
0.5858884758402823 0.3483673469387755
[0.76873866]
[1.3108734]
0.5858884758402823 0.35530612244897963
[0.76479918]
[1.31449433]
0.5858884758402823 0.3622448979591837
[0.76091815]
[1.31807355]
0.5858884758402823 0.3691836734693878
[0.7570947]
[1.32161208]
0.5858884758402823 0.37612244897959185
[0.75332798]
[1.32511088]
0.5858884758402823 0.38306122448979596
[0.74961713]
[1.32857091]
0.5858884758402823 0.39
[1.01458057]
[1.10598209]
0.5959900012858043 0.05
[1.00819472]
[1.11211776]
0.5959900012858043 0.056938775510204084
[1.00182517]
[1.11820797]
0.5959900012858043 0.06387755102040817
[0.99547576]
[1.12424999]
0.5959900012858043 0.07081632653061225
[0.98915026]
[1.1302413]
0.5959900012858043 0.07775510204081633
[0.98285229]
[1.13617957]
0.5959900012858043 0.08469387755102042
[0.97658538]
[1.1420627]
0.5959900012858043 0.091632653061224

[0.91041767]
[1.24134592]
0.6161930521768486 0.19571428571428573
[0.90502873]
[1.24620147]
0.6161930521768486 0.20265306122448984
[0.899702]
[1.25099559]
0.6161930521768486 0.2095918367346939
[0.89443808]
[1.25572881]
0.6161930521768486 0.216530612244898
[0.88923749]
[1.26040175]
0.6161930521768486 0.22346938775510206
[0.88410061]
[1.26501506]
0.6161930521768486 0.23040816326530617
[0.87902771]
[1.26956948]
0.6161930521768486 0.23734693877551022
[0.87401898]
[1.27406579]
0.6161930521768486 0.24428571428571433
[0.8690745]
[1.27850481]
0.6161930521768486 0.2512244897959184
[0.86419425]
[1.28288739]
0.6161930521768486 0.2581632653061225
[0.85937816]
[1.28721443]
0.6161930521768486 0.26510204081632655
[0.85462604]
[1.29148685]
0.6161930521768486 0.27204081632653065
[0.84993768]
[1.29570559]
0.6161930521768486 0.2789795918367347
[0.84531275]
[1.29987161]
0.6161930521768486 0.2859183673469388
[0.8407509]
[1.30398587]
0.6161930521768486 0.29285714285714287
[0.83625171]
[1.30804937]
0.61619305

In [ ]:
np.shape(result)

In [ ]:
end_total = time.time()
print('total time: ',(end_total-start_total)/60.,' minutes')